## Demo of vessel density calculation

In [ ]:
# Import necessary libraries
from liom_toolkit.segmentation import generate_itk_id_list_of_region, create_filter_image, filter_image_to_region, compute_mask_area
from liom_toolkit.utils import load_node_by_name, load_zarr, dask_client_manager

In [ ]:
# Setup dask client.

# Local
dask_client_manager.set_client()

# Remote
#dask_client_manager.set_client("tcp://...")

client = dask_client_manager.get_client()

In [ ]:
# Load the zarr datasets
image_path = ""
nodes = load_zarr(image_path)
# Set the resolution level, 0 is the highest resolution
resolution_level = 0

# Load the atlas, image and mask
atlas_node = load_node_by_name(nodes, "atlas")
atlas = client.scatter(atlas_node.data[resolution_level])

vascular_node = load_node_by_name(nodes, "vessels")
vascular = client.scatter(vascular_node.data[resolution_level])

mask_node = load_node_by_name(nodes, "mask")
mask = client.scatter(mask_node.data[resolution_level])

In [ ]:
# Define the region of interest
region = "Thalamus"

In [ ]:
# Get the region ids which are in the atlas
itk_ids = generate_itk_id_list_of_region(region)

In [ ]:
# Create a filter image
filter_image = create_filter_image(atlas, itk_ids)

# Apply the filter image to the vascular image and the mask
filtered_vascular = filter_image_to_region(vascular, filter_image)
filtered_mask = filter_image_to_region(mask, filter_image)

In [ ]:
# Compute the areas of both the vessel and the mask
vascular_area = compute_mask_area(filtered_vascular)
mask_area = compute_mask_area(filtered_mask)

In [ ]:
# Calculate the vessel density
vessel_density = vascular_area / mask_area
vessel_density